In [1]:
import microsoftvision
import torch

# This will load pretrained model
model = microsoftvision.models.resnet50(pretrained=True,num_classes=1000)

# Load model to CPU memory, interface is the same as torchvision
model = microsoftvision.models.resnet50(pretrained=True, map_location=torch.device('cpu'))

ModuleNotFoundError: No module named 'microsoftvision'

In [ ]:
from torchvision.models.resnet import resnext101_32x8d

In [3]:
import torch
import numpy as np
import pandas as pd

def process_df(df):
    benford_d1 = [np.log10(1 + 1 / 1), np.log10(1 + 1 / 2), np.log10(1 + 1 / 3), np.log10(1 + 1 / 4),
                  np.log10(1 + 1 / 5), np.log10(1 + 1 / 6), np.log10(1 + 1 / 7), np.log10(1 + 1 / 8),
                  np.log10(1 + 1 / 9)]

    df['sumA'] = df['1'] + df['2'] + df['3'] + df['4'] + df['5'] + df['6'] + df['7'] + df['8'] + df['9']
    df['sumB'] = df['-1'] + df['-2'] + df['-3'] + df['-4'] + df['-5'] + df['-6'] + df['-7'] + df['-8'] + df['-9']
    df['sumC'] = df['sumA'] + df['sumB']

    df['log10(parameters)'] = np.log10(df['sumC'])

    df['parameter_category'] = np.int8(np.rint(df['log10(parameters)']))

    df['positive_ratio'] = df['sumA'] / df['sumC']
    df['negative_ratio'] = df['sumB'] / df['sumC']

    df['percent_zero'] = (df['0'] + df['-0']) / (df['sumC'] + df['0'] + df['-0'])

    for z in ['-1', '-2', '-3', '-4', '-5', '-6', '-7', '-8', '-9']:
        df[f'percent_neg{z}'] = df[z] / df['sumB']

        df.loc[df[f'percent_neg{z}'] == 0.0, f'percent_neg{z}'] = 0.00001
        df.fillna(0.00001, inplace=True)

    for z in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
        df[f'percent_{z}'] = df[z] / df['sumA']
        df[f'combined_{z}'] = (df[z] + df['-' + z]) / df['sumC']

        df.loc[df[f'percent_{z}'] == 0.0, f'percent_{z}'] = 0.00001

        df.loc[df[f'combined_{z}'] == 0.0, f'combined_{z}'] = 0.00001

        df.fillna(0.00001, inplace=True)

        if z != '0':
            percent_expected = np.log10(1 + (1 / int(z)))
            df[f'diffpercent_{z}'] = percent_expected - df[f'percent_{z}']
            df[f'diffpercent_neg{z}'] = percent_expected - df[f'percent_neg-{z}']

            print(percent_expected)
            df[f'diffcombined_{z}'] = percent_expected - df[f'combined_{z}']

    df['KL_relative_entropy'] = (df['combined_1'] * np.log(df['combined_1'] / benford_d1[0])) + \
                                (df['combined_2'] * np.log(df['combined_2'] / benford_d1[1])) + \
                                (df['combined_3'] * np.log(df['combined_3'] / benford_d1[2])) + \
                                (df['combined_4'] * np.log(df['combined_4'] / benford_d1[3])) + \
                                (df['combined_5'] * np.log(df['combined_5'] / benford_d1[4])) + \
                                (df['combined_6'] * np.log(df['combined_6'] / benford_d1[5])) + \
                                (df['combined_7'] * np.log(df['combined_7'] / benford_d1[6])) + \
                                (df['combined_8'] * np.log(df['combined_8'] / benford_d1[7])) + \
                                (df['combined_9'] * np.log(df['combined_9'] / benford_d1[8]))

    df['KL_relative_entropy_neg'] = (df['percent_neg-1'] * np.log(df['percent_neg-1'] / benford_d1[0])) + \
                                    (df['percent_neg-2'] * np.log(df['percent_neg-2'] / benford_d1[1])) + \
                                    (df['percent_neg-3'] * np.log(df['percent_neg-3'] / benford_d1[2])) + \
                                    (df['percent_neg-4'] * np.log(df['percent_neg-4'] / benford_d1[3])) + \
                                    (df['percent_neg-5'] * np.log(df['percent_neg-5'] / benford_d1[4])) + \
                                    (df['percent_neg-6'] * np.log(df['percent_neg-6'] / benford_d1[5])) + \
                                    (df['percent_neg-7'] * np.log(df['percent_neg-7'] / benford_d1[6])) + \
                                    (df['percent_neg-8'] * np.log(df['percent_neg-8'] / benford_d1[7])) + \
                                    (df['percent_neg-9'] * np.log(df['percent_neg-9'] / benford_d1[8]))

    df['KL_relative_entropy_pos'] = (df['percent_1'] * np.log(df['percent_1'] / benford_d1[0])) + \
                                    (df['percent_2'] * np.log(df['percent_2'] / benford_d1[1])) + \
                                    (df['percent_3'] * np.log(df['percent_3'] / benford_d1[2])) + \
                                    (df['percent_4'] * np.log(df['percent_4'] / benford_d1[3])) + \
                                    (df['percent_5'] * np.log(df['percent_5'] / benford_d1[4])) + \
                                    (df['percent_6'] * np.log(df['percent_6'] / benford_d1[5])) + \
                                    (df['percent_7'] * np.log(df['percent_7'] / benford_d1[6])) + \
                                    (df['percent_8'] * np.log(df['percent_8'] / benford_d1[7])) + \
                                    (df['percent_9'] * np.log(df['percent_9'] / benford_d1[8]))

    uniform = 1. / 9.

    df['KL_relative_entropy_uniform'] = (df['combined_1'] * np.log(df['combined_1'] / uniform)) + \
                                        (df['combined_2'] * np.log(df['combined_2'] / uniform)) + \
                                        (df['combined_3'] * np.log(df['combined_3'] / uniform)) + \
                                        (df['combined_4'] * np.log(df['combined_4'] / uniform)) + \
                                        (df['combined_5'] * np.log(df['combined_5'] / uniform)) + \
                                        (df['combined_6'] * np.log(df['combined_6'] / uniform)) + \
                                        (df['combined_7'] * np.log(df['combined_7'] / uniform)) + \
                                        (df['combined_8'] * np.log(df['combined_8'] / uniform)) + \
                                        (df['combined_9'] * np.log(df['combined_9'] / uniform)) \
 
    df['KL_relative_entropy_neg_uniform'] = (df['percent_neg-1'] * np.log(df['percent_neg-1'] / uniform)) + \
                                                    (df['percent_neg-2'] * np.log(df['percent_neg-2'] / uniform)) + \
                                                    (df['percent_neg-3'] * np.log(df['percent_neg-3'] / uniform)) + \
                                                    (df['percent_neg-4'] * np.log(df['percent_neg-4'] / uniform)) + \
                                                    (df['percent_neg-5'] * np.log(df['percent_neg-5'] / uniform)) + \
                                                    (df['percent_neg-6'] * np.log(df['percent_neg-6'] / uniform)) + \
                                                    (df['percent_neg-7'] * np.log(df['percent_neg-7'] / uniform)) + \
                                                    (df['percent_neg-8'] * np.log(df['percent_neg-8'] / uniform)) + \
                                                    (df['percent_neg-9'] * np.log(df['percent_neg-9'] / uniform)) \
 
    df['KL_relative_entropy_pos_uniform'] = (df['percent_1'] * np.log(df['percent_1'] / uniform)) + \
                                                    (df['percent_2'] * np.log(df['percent_2'] / uniform)) + \
                                                    (df['percent_3'] * np.log(df['percent_3'] / uniform)) + \
                                                    (df['percent_4'] * np.log(df['percent_4'] / uniform)) + \
                                                    (df['percent_5'] * np.log(df['percent_5'] / uniform)) + \
                                                    (df['percent_6'] * np.log(df['percent_6'] / uniform)) + \
                                                    (df['percent_7'] * np.log(df['percent_7'] / uniform)) + \
                                                    (df['percent_8'] * np.log(df['percent_8'] / uniform)) + \
                                                    (df['percent_9'] * np.log(df['percent_9'] / uniform))

    df['KL_benford_diff'] = df['KL_relative_entropy'].diff()
    df['KL_uniform_diff'] = df['KL_relative_entropy_uniform'].diff()

    df['accuracy_diff'] = df['accuracy'].diff()
    df['loss_diff'] = df['loss'].diff()
    df['lr'] = 0.10
    df['neg_ratio_diff'] = df['negative_ratio'].diff()

    return df

In [ ]:
def calculate_digits(model, model_name, epoch=None, iteration=None, accuracy=None, loss=None):
    # model_name = model.__class__.__name__
    # model.to('cuda:0')

    print('calculate_digits')
    i = 0
    level_counter_list = []
    level_counter2_list = []
    my_counter = {'model': model_name, 'epoch': epoch, 'iteration': iteration, 'accuracy': accuracy, 'loss': loss,
                  '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '-0': 0, '-1': 0,
                  '-2': 0, '-3': 0, '-4': 0, '-5': 0, '-6': 0, '-7': 0, '-8': 0, '-9': 0}

    my_counter2 = {'model': model_name, 'epoch': epoch, 'iteration': iteration, 'accuracy': accuracy, 'loss': loss,
                   '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '6': 0, '8': 0, '9': 0}

    for param in model.parameters():

        print(i)



        level_counter = {'model': model_name, 'row': i, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                         '8': 0, '9': 0, '-0': 0, '-1': 0, '-2': 0, '-3': 0, '-4': 0, '-5': 0, '-6': 0, '-7': 0,
                         '-8': 0, '-9': 0}
        level_counter2 = {'model': model_name, 'row': i, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                          '6': 0, '8': 0, '9': 0}

        np_param = param.cpu().detach().numpy().flatten()
        print(f'len(np_param)={len(np_param)}')

        reps = len(np_param) // 10000

        if len(np_param) % 10000 != 0:
            reps += 1

        for j in range(reps):
            start = j * 10000
            end = start + 10000
            if j == reps - 1:
                section = np_param[start:]
                # print(f'len(section)={len(section)} reps={reps} j={j} npparamlen={len(np_param)}')
            else:
                section = np_param[start:end]

            digits = np.array2string(section, formatter={'float_kind': lambda x: ("%0.1E" % x).split('E')[0]},
                                     threshold=1000000000)

            if len(digits) > 0:
                for d in digits.split(" "):
                    digit = d.replace("\n", "").replace('[', '').replace(']', '')
                    fdigit = digit.split(".")[0]
                    sdigit = digit.split(".")[1]

                    my_counter[fdigit] += 1
                    my_counter2[sdigit] += 1
                    level_counter[fdigit] += 1
                    level_counter2[sdigit] += 1



        level_counter_list.append(level_counter)
        level_counter2_list.append(level_counter2)
        i += 1

    # print(my_counter)
    #level_counter_list, my_counter, level_counter2_list, my_counter2

    print('exporting count.')
    df = pd.DataFrame(level_counter_list)
    df.to_csv(f'lvl_counter_{model_name.split("/")[-1]}.csv')

    df = pd.DataFrame(level_counter2_list)
    df.to_csv(f'lvl_counter2_{model_name.split("/")[-1]}.csv')

    df = pd.DataFrame([my_counter])
    df.to_csv(f'mycounter_{model_name.split("/")[-1]}.csv')

    df = pd.DataFrame([my_counter2])
    df.to_csv(f'mycounter2_{model_name.split("/")[-1]}.csv')

    print('done exporting count.')

In [ ]:
calculate_digits(model,'microsoftvisionresnet50')

In [ ]:
mod2 = torchvision.models.resnet.wide_resnet50_2(pretrained=True, num_classes=1000)
calculate_digits(mod2,'wide_resnet50_2')

In [ ]:
!dir

In [ ]:
df=pd.read_csv('lvl_counter_config_perf2.yaml.csv')
df=process_df(df)

In [ ]:
df[['model','accuracy','log10(parameters)','KL_relative_entropy_uniform','KL_relative_entropy','KL_relative_entropy_neg_uniform','KL_relative_entropy_neg','KL_relative_entropy_pos_uniform','KL_relative_entropy_pos','negative_ratio']]

In [ ]:
df[['model','loss']].plot()

In [ ]:
df[['model','accuracy']].plot()

In [ ]:
df[['model','KL_relative_entropy','KL_relative_entropy_neg','KL_relative_entropy_pos']].plot()

In [ ]:
df[['model','percent_1','percent_2','percent_3','percent_4','percent_5','percent_6','percent_7','percent_8','percent_9']]

In [ ]:
df=pd.read_csv('lvl_counter_config_exp.yaml.csv')
df=process_df(df)

In [ ]:
df.describe()

In [ ]:
df[['model','accuracy','log10(parameters)','KL_relative_entropy_uniform','KL_relative_entropy','KL_relative_entropy_neg_uniform','KL_relative_entropy_neg','KL_relative_entropy_pos_uniform','KL_relative_entropy_pos','negative_ratio']]

In [ ]:
both=df[['1','2','3','4','5','6','7','8','9']].values + df[['-1','-2','-3','-4','-5','-6','-7','-8','-9']].values

In [ ]:
np.sum(both)

In [ ]:
np.sum(ideal) 

In [ ]:
np.sum(np.abs(both - ideal))/np.sum(both)

In [ ]:
np.sum(np.abs(both - ideal))

In [ ]:
both - ideal

In [ ]:
ideal

In [ ]:
ideal=np.sum(both) * np.array([np.log10(1 + 1 / 1), np.log10(1 + 1 / 2), np.log10(1 + 1 / 3), np.log10(1 + 1 / 4),
                  np.log10(1 + 1 / 5), np.log10(1 + 1 / 6), np.log10(1 + 1 / 7), np.log10(1 + 1 / 8),
                  np.log10(1 + 1 / 9)]) 

In [ ]:
df[['1','2','3','4','5','6','7','8','9']]

In [ ]:
df[['model','percent_1','percent_2','percent_3','percent_4','percent_5','percent_6','percent_7','percent_8','percent_9']]

In [ ]:
df[['model','percent_1','percent_2','percent_3','percent_4','percent_5','percent_6','percent_7','percent_8','percent_9']].plot()

In [ ]:
import matplotlib.pyplot as plt


plt.plot(df[['percent_1','percent_2','percent_3','percent_4','percent_5','percent_6','percent_7','percent_8','percent_9']].values.flatten(),np.arange(1,10))

In [ ]:
df[['percent_1','percent_2','percent_3','percent_4','percent_5','percent_6','percent_7','percent_8','percent_9']].values

In [ ]:
df[['model','percent_neg-1','percent_neg-2','percent_neg-3','percent_neg-4','percent_neg-5','percent_neg-6','percent_neg-7','percent_neg-8','percent_neg-9']]

In [ ]:
plt.plot(df[['percent_neg-1','percent_neg-2','percent_neg-3','percent_neg-4','percent_neg-5','percent_neg-6','percent_neg-7','percent_neg-8','percent_neg-9']].values.flatten(),np.arange(1,10))

At random initialization

In [ ]:


# Load model to CPU memory, interface is the same as torchvision
model2 = microsoftvision.models.resnet50(pretrained=False, map_location=torch.device('cpu'))

In [ ]:
calculate_digits(model2,'microsoftvisionresnet50_init')

In [ ]:
df2=pd.read_csv('mycounter_microsoftvisionresnet50_init.csv')
df2=process_df(df2)

In [ ]:
df2[['model','accuracy','log10(parameters)','KL_relative_entropy_uniform','KL_relative_entropy','KL_relative_entropy_neg_uniform','KL_relative_entropy_neg','KL_relative_entropy_pos_uniform','KL_relative_entropy_pos','negative_ratio']]

In [ ]:
df[['model','accuracy','log10(parameters)','KL_relative_entropy_uniform','KL_relative_entropy','KL_relative_entropy_neg_uniform','KL_relative_entropy_neg','KL_relative_entropy_pos_uniform','KL_relative_entropy_pos','negative_ratio']]

In [ ]:
!pip install torchbench

In [ ]:
from torchbench.image_classification import ImageNet

In [ ]:
#from torchvision.models.resnet import resnext101_32x8d
import torchvision.transforms as transforms
import PIL

# Define the transforms need to convert ImageNet data to expected model input
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
    std=[0.229, 0.224, 0.225])
input_transform = transforms.Compose([
    transforms.Resize(256, PIL.Image.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

# Run the benchmark
ImageNet.benchmark(
    model=resnext101_32x8d(pretrained=True, num_classes=1000), #microsoftvision.models.resnet50(pretrained=True)
    paper_model_name='microsoftvision50',
  
    input_transform=input_transform,
    batch_size=256,
    num_gpu=1
)

In [ ]:
!cd ".data/vision"
!mkdir imagenet 

In [ ]:
import torchvision

In [ ]:
dataset=torchvision.datasets.ImageNet(".data/vision/imagenet")

In [5]:
def process_df(df):
    benford_d1 = [np.log10(1 + 1 / 1), np.log10(1 + 1 / 2), np.log10(1 + 1 / 3), np.log10(1 + 1 / 4),
                  np.log10(1 + 1 / 5), np.log10(1 + 1 / 6), np.log10(1 + 1 / 7), np.log10(1 + 1 / 8),
                  np.log10(1 + 1 / 9)]

    df['sumA'] = df['1'] + df['2'] + df['3'] + df['4'] + df['5'] + df['6'] + df['7'] + df['8'] + df['9']
    df['sumB'] = df['-1'] + df['-2'] + df['-3'] + df['-4'] + df['-5'] + df['-6'] + df['-7'] + df['-8'] + df['-9']
    df['sumC'] = df['sumA'] + df['sumB']

    df['log10(parameters)'] = np.log10(df['sumC'])

    df['parameter_category'] = np.int8(np.rint(df['log10(parameters)']))

    df['positive_ratio'] = df['sumA'] / df['sumC']
    df['negative_ratio'] = df['sumB'] / df['sumC']

    df['percent_zero'] = (df['0'] + df['-0']) / (df['sumC'] + df['0'] + df['-0'])

    for z in ['-1', '-2', '-3', '-4', '-5', '-6', '-7', '-8', '-9']:
        df[f'percent_neg{z}'] = df[z] / df['sumB']

        df.loc[df[f'percent_neg{z}'] == 0.0, f'percent_neg{z}'] = 0.00001
        df.fillna(0.00001, inplace=True)

    for z in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
        df[f'percent_{z}'] = df[z] / df['sumA']
        df[f'combined_{z}'] = (df[z] + df['-' + z]) / df['sumC']

        df.loc[df[f'percent_{z}'] == 0.0, f'percent_{z}'] = 0.00001

        df.loc[df[f'combined_{z}'] == 0.0, f'combined_{z}'] = 0.00001

        df.fillna(0.00001, inplace=True)

        if z != '0':
            percent_expected = np.log10(1 + (1 / int(z)))
            df[f'diffpercent_{z}'] = percent_expected - df[f'percent_{z}']
            df[f'diffpercent_neg{z}'] = percent_expected - df[f'percent_neg-{z}']

            print(percent_expected)
            df[f'diffcombined_{z}'] = percent_expected - df[f'combined_{z}']

    df['KL_relative_entropy'] = (df['combined_1'] * np.log(df['combined_1'] / benford_d1[0])) + \
                                (df['combined_2'] * np.log(df['combined_2'] / benford_d1[1])) + \
                                (df['combined_3'] * np.log(df['combined_3'] / benford_d1[2])) + \
                                (df['combined_4'] * np.log(df['combined_4'] / benford_d1[3])) + \
                                (df['combined_5'] * np.log(df['combined_5'] / benford_d1[4])) + \
                                (df['combined_6'] * np.log(df['combined_6'] / benford_d1[5])) + \
                                (df['combined_7'] * np.log(df['combined_7'] / benford_d1[6])) + \
                                (df['combined_8'] * np.log(df['combined_8'] / benford_d1[7])) + \
                                (df['combined_9'] * np.log(df['combined_9'] / benford_d1[8]))

    df['KL_relative_entropy_neg'] = (df['percent_neg-1'] * np.log(df['percent_neg-1'] / benford_d1[0])) + \
                                    (df['percent_neg-2'] * np.log(df['percent_neg-2'] / benford_d1[1])) + \
                                    (df['percent_neg-3'] * np.log(df['percent_neg-3'] / benford_d1[2])) + \
                                    (df['percent_neg-4'] * np.log(df['percent_neg-4'] / benford_d1[3])) + \
                                    (df['percent_neg-5'] * np.log(df['percent_neg-5'] / benford_d1[4])) + \
                                    (df['percent_neg-6'] * np.log(df['percent_neg-6'] / benford_d1[5])) + \
                                    (df['percent_neg-7'] * np.log(df['percent_neg-7'] / benford_d1[6])) + \
                                    (df['percent_neg-8'] * np.log(df['percent_neg-8'] / benford_d1[7])) + \
                                    (df['percent_neg-9'] * np.log(df['percent_neg-9'] / benford_d1[8]))

    df['KL_relative_entropy_pos'] = (df['percent_1'] * np.log(df['percent_1'] / benford_d1[0])) + \
                                    (df['percent_2'] * np.log(df['percent_2'] / benford_d1[1])) + \
                                    (df['percent_3'] * np.log(df['percent_3'] / benford_d1[2])) + \
                                    (df['percent_4'] * np.log(df['percent_4'] / benford_d1[3])) + \
                                    (df['percent_5'] * np.log(df['percent_5'] / benford_d1[4])) + \
                                    (df['percent_6'] * np.log(df['percent_6'] / benford_d1[5])) + \
                                    (df['percent_7'] * np.log(df['percent_7'] / benford_d1[6])) + \
                                    (df['percent_8'] * np.log(df['percent_8'] / benford_d1[7])) + \
                                    (df['percent_9'] * np.log(df['percent_9'] / benford_d1[8]))

    uniform = 1. / 9.

    df['KL_relative_entropy_uniform'] = (df['combined_1'] * np.log(df['combined_1'] / uniform)) + \
                                        (df['combined_2'] * np.log(df['combined_2'] / uniform)) + \
                                        (df['combined_3'] * np.log(df['combined_3'] / uniform)) + \
                                        (df['combined_4'] * np.log(df['combined_4'] / uniform)) + \
                                        (df['combined_5'] * np.log(df['combined_5'] / uniform)) + \
                                        (df['combined_6'] * np.log(df['combined_6'] / uniform)) + \
                                        (df['combined_7'] * np.log(df['combined_7'] / uniform)) + \
                                        (df['combined_8'] * np.log(df['combined_8'] / uniform)) + \
                                        (df['combined_9'] * np.log(df['combined_9'] / uniform)) \
 
    df['KL_relative_entropy_neg_uniform'] = (df['percent_neg-1'] * np.log(df['percent_neg-1'] / uniform)) + \
                                                    (df['percent_neg-2'] * np.log(df['percent_neg-2'] / uniform)) + \
                                                    (df['percent_neg-3'] * np.log(df['percent_neg-3'] / uniform)) + \
                                                    (df['percent_neg-4'] * np.log(df['percent_neg-4'] / uniform)) + \
                                                    (df['percent_neg-5'] * np.log(df['percent_neg-5'] / uniform)) + \
                                                    (df['percent_neg-6'] * np.log(df['percent_neg-6'] / uniform)) + \
                                                    (df['percent_neg-7'] * np.log(df['percent_neg-7'] / uniform)) + \
                                                    (df['percent_neg-8'] * np.log(df['percent_neg-8'] / uniform)) + \
                                                    (df['percent_neg-9'] * np.log(df['percent_neg-9'] / uniform)) \
 
    df['KL_relative_entropy_pos_uniform'] = (df['percent_1'] * np.log(df['percent_1'] / uniform)) + \
                                                    (df['percent_2'] * np.log(df['percent_2'] / uniform)) + \
                                                    (df['percent_3'] * np.log(df['percent_3'] / uniform)) + \
                                                    (df['percent_4'] * np.log(df['percent_4'] / uniform)) + \
                                                    (df['percent_5'] * np.log(df['percent_5'] / uniform)) + \
                                                    (df['percent_6'] * np.log(df['percent_6'] / uniform)) + \
                                                    (df['percent_7'] * np.log(df['percent_7'] / uniform)) + \
                                                    (df['percent_8'] * np.log(df['percent_8'] / uniform)) + \
                                                    (df['percent_9'] * np.log(df['percent_9'] / uniform))

    df['KL_benford_diff'] = df['KL_relative_entropy'].diff()
    df['KL_uniform_diff'] = df['KL_relative_entropy_uniform'].diff()

    #df['accuracy_diff'] = df['accuracy'].diff()
    #df['loss_diff'] = df['loss'].diff()
    #df['lr'] = 0.10
    df['neg_ratio_diff'] = df['negative_ratio'].diff()

    return df

In [11]:
df2=pd.read_csv('lvl_counter_RN50X4.csv')
df2=process_df(df2)

0.3010299956639812
0.17609125905568124
0.12493873660829992
0.09691001300805642
0.07918124604762482
0.06694678963061322
0.05799194697768673
0.05115252244738129
0.04575749056067514


/home/dlesser/anaconda3/envs/CLIP/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
df2[['KL_relative_entropy_uniform','KL_relative_entropy','KL_relative_entropy_neg_uniform','KL_relative_entropy_neg','KL_relative_entropy_pos_uniform','KL_relative_entropy_pos','negative_ratio']]

KL_relative_entropy_uniform  KL_relative_entropy  \
0                       0.169325             0.008207   
1                       0.138800             0.011972   
2                       0.283476             0.019609   
3                       0.200720             0.001491   
4                       0.138171             0.184142   
..                           ...                  ...   
150                     1.408587             2.054815   
151                     0.237963             0.043079   
152                     0.288207             0.022208   
153                     1.574675             2.025698   
154                     0.388340             0.045298   

     KL_relative_entropy_neg_uniform  KL_relative_entropy_neg  \
0                           0.174212                 0.011372   
1                           0.112165                 0.028430   
2                           0.283766                 0.019685   
3                           0.199582                 0.001523   
4                           0.769791                 0.351003   
..                               ...                      ...   
150                        -0.000838                -0.000821   
151                         0.348804                 0.063667   
152                         0.288296                 0.022082   
153                        -0.000838                -0.000821   
154                         0.308629                 0.032867   

     KL_relative_entropy_pos_uniform  KL_relative_entropy_pos  negative_ratio  
0                           0.165050                 0.007017        0.641927  
1                           0.208398                 0.000374        0.653281  
2                           0.283200                 0.019547        0.499466  
3                           0.201865                 0.001467        0.500038  
4                           0.139482                 0.190966        0.011719  
..                               ...                      ...             ...  
150                         1.408587                 2.054815        0.000000  
151                         0.214008                 0.044976        0.222656  
152                         0.288118                 0.022335        0.500141  
153                         1.574675                 2.025698        0.000000  
154                         0.425131                 0.059662        0.250000  

[155 rows x 7 columns]

In [12]:
df2.to_csv('benford_rn50x4.csv')